### Data Preparation and Uploading to the Database

This notebook deals with loading data from a CSV file into a SQL database as part of the Exploratory Data Analysis (EDA) process. Transformations such as column name capitalization will be performed to improve the readability of the data before loading it into the database.

Set the work envarioment

In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv("WORK_DIR")

sys.path.append(work_dir)


These imports allow us to interact with the database and perform operations such as querying and manipulating Steam game data effectively.

In [3]:
import pandas as pd
from src.models.SteamGames_models import Games
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from src.database.db_connection import get_engine

Connection and sessions

In [4]:
connection = get_engine()

Session = sessionmaker(bind=connection)
session = Session()

Conected successfully to database PruebaCarga!


This code ensures that the table 'raw_games' is updated in the database, using the Games model to do so.

In [5]:
if inspect(connection).has_table('raw_games'):
    try:
        Games.__table__.drop(connection)
        Games.__table__.create(connection)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")
else:
    try:
        Games.__table__.create(connection)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

Table created successfully.


Transformation of column names 

In [6]:
games = pd.read_csv('../data/games.csv')

def capitalize_next_word(column_name):
    result = ""
    capitalize_next = True
    for char in column_name:
        if char == " ":
            capitalize_next = True
        elif capitalize_next:
            result += char.upper()
            capitalize_next = False
        else:
            result += char
    return result

games.rename(columns=capitalize_next_word, inplace=True)
games.columns

Index(['AppID', 'Name', 'ReleaseDate', 'EstimatedOwners', 'PeakCCU',
       'RequiredAge', 'Price', 'DLCCount', 'AboutTheGame',
       'SupportedLanguages', 'FullAudioLanguages', 'Reviews', 'HeaderImage',
       'Website', 'SupportUrl', 'SupportEmail', 'Windows', 'Mac', 'Linux',
       'MetacriticScore', 'MetacriticUrl', 'UserScore', 'Positive', 'Negative',
       'ScoreRank', 'Achievements', 'Recommendations', 'Notes',
       'AveragePlaytimeForever', 'AveragePlaytimeTwoWeeks',
       'MedianPlaytimeForever', 'MedianPlaytimeTwoWeeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

Uploading Information to the Database

In [7]:
try:
    games.to_sql('raw_games', connection, if_exists='replace', index=False)
    print("Data uploaded")

except Exception as e:
    print(f"Error: {e}")
    
finally:
    session.close()

Data uploaded


**Here ends the first notebook of this project, you must continue with notebook 002.**